![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [4]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK", "*.oc"))

# Totem
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast", "*.oc"))

# Gather
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather", "*.oc"))

# Blueberry 2024
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "BLUEBERRY_FESTIVAL_24"))

# Harvest 2024
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "HARVEST_FESTIVAL_24"))

#Mercante 
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "MERCANTE", "*.oc"))

#Sage Catering
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "SAGE_CATERING", "*.oc"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/SAGE_CATERING/SageCatering_26Nov.oc',
 '/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/SAGE_CATERING/SageCatering_12thNov.oc']

### Import Items List

In [5]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/Items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )

Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

In [6]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [7]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [8]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [9]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

[]

In [10]:
Items = Items[~Items["InventoryGroup"].isin(["DISPOSABLES", "SUPPLY"])]

In [11]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(220, 7)

In [12]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [13]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [14]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [15]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [16]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

23      I-3660
26      I-4657
27      I-5983
35      I-4657
39      I-3660
        ...   
578    I-71959
580     I-1109
581     I-3434
588     I-1109
590    I-71937
Name: IngredientId, Length: 310, dtype: object

In [17]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [18]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3386,500.000,ml,0.00100000,500.0000,P-12957
1,I-4657,1.250,lb,1.00000000,1.2500,P-12957
2,I-4695,1.000,lb,1.00000000,2.5000,P-12983
3,I-2254,1.000,L,1.00000000,0.6667,P-14877
4,I-3348,0.500,L,1.00000000,0.3333,P-14877
...,...,...,...,...,...,...
586,I-72653,1.000,ea,1.00000000,1.0000,R-74608
587,I-29692,1.000,ea,1.00000000,1.0000,R-74650
588,I-1109,1.000,CT,1.00000000,1.0000,R-74699
589,I-2306,32.000,g,1.00000000,1.0000,R-74699


In [19]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [20]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4473,AVOCADO (20CT) MX,20.000,CT,1.000,ea,PRODUCE
1,I-28775,BAGUETTE BUN*DEMI 150G,1.000,cs,50.000,ea,BREAD
2,I-3619,BAY LEAF WHL SHAKER TFC,12.000,ea,60.000,g,SPICES
3,I-71746,BEANS BLACK*,24.000,can,540.000,ml,FOOD - GROCERY
4,I-4288,BEETS RED,5.000,lb,1.000,lb,PRODUCE
...,...,...,...,...,...,...,...
215,I-61420,TOFU FIRM*MEDIUM,12.000,pak,454.000,g,FOOD - GROCERY
216,I-68648,TOPPING CHOC*SYRP EZ FLOW,8.000,bag,1.500,L,FOOD - GROCERY
217,I-1143,TORTILLA FLOUR*12IN,6.000,bag,10.000,CT,BAKED GOODS
218,I-3986,TURKEY BREAST BONELESS -FRESH,1.000,Kg,1.000,Kg,POULTRY


In [21]:
Ingredients.shape

(591, 6)

In [22]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [24]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-74330,DRESSING|Garlic Tahini Dressin,750.000,ml,
1,P-71838,DRESSING|Goma Mayo,3.000,L,
2,P-74391,DRESSING|Lemon Thyme,4.193,Kg,
3,P-70074,DRESSING|Sesame Ginger Vin,1.695,L,PREP
4,P-74713,DRESSING|Tamarind Cumin Vin,0.920,L,
...,...,...,...,...,...
56,P-74597,SAUCE|Pesto Mayo,1.000,Kg,
57,P-74673,SMD24|PREP|Brined Turkey,5.000,Kg,
58,P-74674,SMD24|PREP|Roasted Turkey,850.000,g,
59,P-70366,SMD24|YIELD|Red Cabbage,765.000,g,


In [26]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [27]:
# There are 546 rows and 5 columns
Preps.shape

(61, 5)

In [28]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup


In [29]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [30]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [31]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [32]:
Products

,ProdId,Description,SalesGroup
0,R-74333,SMD24|BOWL|Lemongrass Chicken,FOOD
1,R-74334,SMD24|BOWL|Lemongrass Prawn,FOOD
2,R-74332,SMD24|BOWL|Lemongrass Tofu,FOOD
3,R-74329,SMD24|BOWL|Pacific Delight,FOOD
4,R-74599,SMD24|BOWL|Salmon Poke,FOOD
5,R-74331,SMD24|BOWL|Salutation Bowl,FOOD
6,R-74600,SMD24|BOWL|Tuna Poke,FOOD
7,R-74441,SMD24|Dessert|Blueberry Tart,FOOD
8,R-74442,SMD24|DESSERT|Lemon Tart,FOOD
9,R-74410,SMD24|LP|Apollo,FOOD


In [33]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup


In [34]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [35]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup


In [36]:
Products.shape

(45, 3)

In [37]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
153,I-4450,0.01000000,1.0000,CT,100.0000,g
154,I-4844,0.00159744,1.0000,HEAD,626.0000,g
155,I-6812,0.00909091,1.0000,cup,110.0000,g
156,I-43543,0.04347826,1.0000,leaf,23.0000,g


In [41]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [42]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-4473' 'I-28775' 'I-3619' 'I-71746' 'I-4288' 'I-4527' 'I-2574' 'I-6020'
 'I-4551' 'I-4557' 'I-4560' 'I-3159' 'I-11214' 'I-4564' 'I-4579' 'I-45827'
 'I-3621' 'I-4599' 'I-71865' 'I-2055' 'I-71937' 'I-71959' 'I-8236'
 'I-2129' 'I-4009' 'I-4007' 'I-4008' 'I-3622' 'I-3623' 'I-3624' 'I-3625'
 'I-3628' 'I-3629' 'I-11094' 'I-4978' 'I-72624' 'I-2217' 'I-4626' 'I-6832'
 'I-4634' 'I-4637' 'I-3634' 'I-71845' 'I-3284' 'I-19631' 'I-6428'
 'I-71499' 'I-11207' 'I-6448' 'I-4657' 'I-3632' 'I-4658' 'I-9685' 'I-3079'
 'I-4671' 'I-4679' 'I-2308' 'I-17675' 'I-18760' 'I-4508' 'I-4620' 'I-4341'
 'I-4695' 'I-10117' 'I-4982' 'I-4767' 'I-4772' 'I-4919' 'I-3321' 'I-4869'
 'I-4697' 'I-1813' 'I-1758' 'I-1759' 'I-2599' 'I-6876' 'I-4463' 'I-4677'
 'I-4845' 'I-5003' 'I-3348' 'I-3351' 'I-4691' 'I-3370' 'I-2898' 'I-1028'
 'I-74044' 'I-74471' 'I-3386' 'I-3387' 'I-3389' 'I-71926' 'I-3395'
 'I-6829' 'I-2910' 'I-3397' 'I-3639' 'I-4678' 'I-10482' 'I-4757' 'I-6807'
 'I-4381' 'I-3314' 'I-2911' 'I-3641' 

In [43]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

155


In [44]:
Conversions.shape

(158, 6)

In [45]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [46]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [47]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [48]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,220,7
Preps,61,5
Ingredients,591,6
Products,45,3
Conversions,158,6
